<a href="https://colab.research.google.com/github/Snake-AID/SnakeAid.ComputerVision/blob/main/SnakeTraining_V4_YOLOv12_Khiem_Bbox5000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment setup

In [ ]:
!nvidia-smi

In [ ]:
import os
from datetime import datetime, timedelta, timezone

HOME = os.getcwd()
print(f"📂 HOME: {HOME}")

PROJECT_NAME = "SnakeTraining"
VERSION      = "V4"
MODEL_ARCH   = "YOLOv12"
AUTHOR       = "Khiem"
DATA_INFO    = "Bbox5000"

# Lấy thời gian hiện tại theo múi giờ UTC+7
utc_plus_7 = timezone(timedelta(hours=7))
current_time = datetime.now(utc_plus_7).strftime("%Y%m%d_%H%M")

# Ghép chuỗi tên model kèm timestamp
notebook_name = f"{PROJECT_NAME}_{VERSION}_{MODEL_ARCH}_{AUTHOR}_{DATA_INFO}_{current_time}"

print(f"📘 Configured Model Name: {notebook_name}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Define Dataset Paths

In [ ]:
import os
import shutil
import yaml
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor

# --- CONFIG ---
DRIVE_DATASET_PATH = '/content/drive/MyDrive/SnakeDataset/SnakeAid-YOLOv12-5000bbox'
DATASET_PATH = '/content/snake_dataset'
DATA_YAML_PATH = f'{DATASET_PATH}/data.yaml'

def check_dataset_integrity(path):
    """Kiểm tra thông minh: Trả về True nếu data đã đầy đủ"""
    # 1. Check file cấu hình
    if not os.path.exists(f"{path}/data.yaml"):
        return False

    # 2. Check các folder quan trọng (train, valid, test)
    # Nếu bất kỳ folder nào thiếu hoặc rỗng -> coi như lỗi -> copy lại
    required_dirs = [
        'train/images', 'train/labels',
        'valid/images', 'valid/labels',
        'test/images', 'test/labels'
    ]

    for d in required_dirs:
        dir_path = os.path.join(path, d)
        if not os.path.exists(dir_path):
            return False
        if not os.listdir(dir_path): # Folder rỗng
            return False

    return True

def copy_file_worker(args):
    """Hàm worker để copy 1 file (dùng cho đa luồng)"""
    src, dst = args
    try:
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copy2(src, dst)
    except Exception as e:
        # Có thể log error nếu cần
        pass

def fast_copy_multithread(src_root, dst_root, workers=16):
    """Copy dữ liệu sử dụng đa luồng để tối ưu I/O"""
    if os.path.exists(dst_root):
        print(f"🗑️ Phát hiện dữ liệu không toàn vẹn. Đang xóa {dst_root}...")
        shutil.rmtree(dst_root)

    print("🔍 Đang quét danh sách file nguồn (Scanning)...")
    files_to_copy = []
    for root, dirs, files in os.walk(src_root):
        for file in files:
            src_file = os.path.join(root, file)
            rel_path = os.path.relpath(src_file, src_root)
            dst_file = os.path.join(dst_root, rel_path)
            files_to_copy.append((src_file, dst_file))

    print(f"🚀 Tìm thấy {len(files_to_copy)} files. Bắt đầu copy tốc độ cao (16 threads)...")

    # ThreadPoolExecutor giúp request nhiều file cùng lúc từ Drive
    with ThreadPoolExecutor(max_workers=workers) as executor:
        list(tqdm(executor.map(copy_file_worker, files_to_copy), total=len(files_to_copy), unit="file"))

# --- MAIN FLOW ---
if check_dataset_integrity(DATASET_PATH):
    print("✅ Smart Check: Dữ liệu đã đầy đủ và hợp lệ tại Local. (Skipped Copy)")
else:
    print(f"⚠️ Smart Check: Dữ liệu thiếu hoặc chưa có. Tiến hành copy...")
    try:
        fast_copy_multithread(DRIVE_DATASET_PATH, DATASET_PATH)

        # Cập nhật đường dẫn trong data.yaml
        if os.path.exists(DATA_YAML_PATH):
            with open(DATA_YAML_PATH, 'r') as f:
                data_config = yaml.safe_load(f)

            data_config['path'] = DATASET_PATH
            # Fix đường dẫn relative
            for key in ['train', 'val', 'test']:
                if key in data_config and isinstance(data_config[key], str):
                    if 'drive' in data_config[key].lower():
                        folder = 'valid' if key == 'val' else key
                        data_config[key] = f'{folder}/images'

            with open(DATA_YAML_PATH, 'w') as f:
                yaml.dump(data_config, f)
            print("✅ Cấu hình data.yaml hoàn tất!")
        else:
            print("⚠️ Không tìm thấy data.yaml sau khi copy!")

    except Exception as e:
        print(f"❌ Lỗi khi copy: {e}")

In [ ]:
from rich import print
from rich.panel import Panel
from rich.table import Table
from rich import box
import os

def check_dataset(root, folders=("train", "valid", "test")):
    table = Table(
        title="Dataset Structure",
        box=box.SIMPLE_HEAD,
        show_lines=False,
        pad_edge=False
    )

    table.add_column("Component", style="bold cyan", justify="center")
    table.add_column("Status/Images", justify="center")
    table.add_column("Labels", justify="center")

    missing_files = False

    yaml_path = os.path.join(root, "data.yaml")
    yaml_exists = os.path.exists(yaml_path)

    if not yaml_exists:
        missing_files = True

    table.add_row(
        "data.yaml",
        "[green]FOUND" if yaml_exists else "[red]MISSING",
        "-"
    )

    for folder in folders:
        fp = os.path.join(root, folder)
        img = os.path.join(fp, "images")
        lbl = os.path.join(fp, "labels")

        img_exists = os.path.exists(img)
        lbl_exists = os.path.exists(lbl)

        if not img_exists or not lbl_exists:
            missing_files = True

        table.add_row(
            folder.upper(),
            "[green]FOUND" if img_exists else "[red]MISSING",
            "[green]FOUND" if lbl_exists else "[red]MISSING",
        )

    panel = Panel.fit(
        table,
        title="🐍 Snake Dataset Check",
        border_style="cyan",
        box=box.SIMPLE
    )

    print(panel)

    if missing_files:
        raise FileNotFoundError("Dataset structure is incomplete! Please check your dataset path and structure.")

check_dataset(DATASET_PATH)

# Install YOLOv12 and SuperVision

## Install essential python package

In [ ]:
!pip install -q git+https://github.com/sunsmarterjie/yolov12.git supervision

# Test the base models

### Download example data

In [ ]:
!wget https://media.roboflow.com/notebooks/examples/dog.jpeg

### Run inference

In the example, we're using the `yolov12l.pt` model, but you can experiment with different model sizes by simply swapping out the model name during initialization. Options include `yolov12n.pt`, `yolov12s.pt`, `yolov12m.pt`, `yolov12l.pt`, and `yolov12x.pt`.

In [ ]:
import cv2
from ultralytics import YOLO
import supervision as sv


image_path = f"{HOME}/dog.jpeg"
image = cv2.imread(image_path)

model = YOLO('yolov12l.pt')

results = model(image, verbose=False)[0]
detections = sv.Detections.from_ultralytics(results)

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = image.copy()
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)

# Fine-tune YOLOv12 model

We are now ready to fine-tune our YOLOv12 model. In the code below, we initialize the model using a starting checkpoint—here, we use `yolov12s.yaml`, but you can replace it with any other model (e.g., `yolov12n.pt`, `yolov12m.pt`, `yolov12l.pt`, or `yolov12x.pt`) based on your preference. We set the training to run for 100 epochs in this example; however, you should adjust the number of epochs along with other hyperparameters such as batch size, image size, and augmentation settings (scale, mosaic, mixup, and copy-paste) based on your hardware capabilities and dataset size.

**Note:** **Note that after training, you might encounter a `TypeError: argument of type 'PosixPath' is not iterable error` — this is a known issue, but your model weights will still be saved, so you can safely proceed to running inference.**

In [ ]:
# from ultralytics import YOLO

# model = YOLO('yolov12m.pt')
# results = model.train(data=DATA_YAML_PATH, epochs=100, patience=10, cache=True)

### Param Config
Chúng ta fine-tune YOLO từ checkpoint pretrained (`yolov12m.pt`) thay vì train từ đầu bằng `.yaml`, nhằm tận dụng transfer learning để đạt kết quả tốt và ổn định hơn cho bài toán nhận diện rắn.

---

#### `epochs=200`

* **Chức năng:** Xác định số vòng lặp huấn luyện tối đa mà mô hình sẽ chạy trên toàn bộ tập dữ liệu.
* **Lý do lựa chọn:** Với đối tượng rắn thường nhỏ và khó phân biệt với nền, các chỉ số như mAP và Recall thường cải thiện muộn, vì vậy cần nhiều epoch hơn mức mặc định (100) để mô hình hội tụ đầy đủ.

---

#### `patience=40`

* **Chức năng:** Điều khiển cơ chế early stopping, mô hình sẽ dừng huấn luyện nếu metric validation không cải thiện sau số epoch này.
* **Lý do lựa chọn:** Trong bài toán SnakeAid, metric thường dao động mạnh do vật thể nhỏ và nền phức tạp, nên đặt patience cao để tránh dừng huấn luyện quá sớm khi mô hình chưa đạt trạng thái tối ưu.

---

#### `imgsz=896`

* **Chức năng:** Thiết lập độ phân giải ảnh đầu vào cho quá trình huấn luyện.
* **Lý do lựa chọn:** Rắn trong ảnh thường chiếm rất ít pixel, tăng độ phân giải giúp mô hình giữ lại nhiều chi tiết không gian hơn, từ đó cải thiện khả năng phát hiện rắn nhỏ hoặc ở xa.

---

#### `batch=16`

* **Chức năng:** Xác định số lượng ảnh được xử lý trong một bước cập nhật gradient.
* **Lý do lựa chọn:** Batch size vừa phải giúp cân bằng giữa độ ổn định của gradient và khả năng học các vật thể nhỏ, tránh hiện tượng làm mượt quá mức khi batch quá lớn.

---

#### `cache=True`

* **Chức năng:** Lưu dữ liệu huấn luyện vào bộ nhớ đệm để tăng tốc độ đọc dữ liệu.
* **Lý do lựa chọn:** Dataset được lưu trên Google Drive khi train bằng Colab, việc bật cache giúp giảm độ trễ I/O và rút ngắn thời gian huấn luyện tổng thể.

---

#### `workers=8`

* **Chức năng:** Xác định số tiến trình CPU song song được sử dụng để đọc dữ liệu, tiền xử lý ảnh và áp dụng các augmentation trước khi dữ liệu được đưa vào GPU trong quá trình huấn luyện.
* **Lý do lựa chọn:** Với ảnh độ phân giải cao (`imgsz=896`) và dữ liệu lưu trên Google Drive khi huấn luyện bằng Colab A100, việc sử dụng `workers=8` giúp tận dụng hiệu quả tài nguyên CPU, tránh tình trạng GPU phải chờ dữ liệu và đảm bảo tốc độ huấn luyện ổn định cho bài toán nhận diện rắn nhỏ và ở xa.

---

#### `cos_lr=True`

* **Chức năng:** Sử dụng chiến lược giảm learning rate theo hàm cosine trong quá trình huấn luyện.
* **Lý do lựa chọn:** Cosine learning rate giúp fine-tune mượt hơn ở các epoch sau, giảm nguy cơ dao động mạnh và giúp mô hình ổn định khi tối ưu cho các vật thể nhỏ.

---

#### `mixup=0.0`

* **Chức năng:** Điều chỉnh mức độ sử dụng augmentation mixup (trộn hai ảnh và nhãn).
* **Lý do lựa chọn:** Mixup có thể tạo ra các hình ảnh không thực tế đối với đối tượng dài và mảnh như rắn, làm giảm chất lượng học, vì vậy được tắt hoàn toàn trong bài toán này.

---

#### `copy_paste=0.0`

* **Chức năng:** Điều khiển augmentation copy-paste, trong đó đối tượng được cắt và dán sang ảnh khác.
* **Lý do lựa chọn:** Copy-paste có thể tạo ra các vị trí rắn không tự nhiên trong ảnh, gây nhiễu cho mô hình, nên được tắt để giữ tính thực tế của dữ liệu huấn luyện.

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov12m.pt")

results = model.train(
    data=DATA_YAML_PATH,
    project=f'{HOME}/runs/detect',
    name='train',
    exist_ok=True,    # Quan trọng: Ghi đè folder cũ để các cell đánh giá bên dưới không bị lỗi đường dẫn

    epochs=17,
    # patience=40,
    imgsz=896,        # Quan trọng cho rắn nhỏ/ở xa
    batch=16,         # Phù hợp với A100. Nếu dùng T4 (Free Colab) hãy giảm xuống 8 nếu bị OOM.
    cache=True,       # Cache vào RAM giúp train cực nhanh (hiệu quả nhất khi data đã ở Local)
    workers=8,
    cos_lr=True,

    # Tắt các augmentation không thực tế với hình dáng rắn
    mixup=0.0,
    copy_paste=0.0,
)

### Save model file into google drive

In [ ]:
import shutil
import os

# Define the source path of the best trained model
source_path = f'{HOME}/runs/detect/train/weights/best.pt'

# Define the destination directory in Google Drive
destination_dir = '/content/drive/MyDrive/SnakeAIModels'
os.makedirs(destination_dir, exist_ok=True)

# Sử dụng biến 'notebook_name' đã được define ở cell Setup đầu notebook
print(f"💾 Saving model for notebook: {notebook_name}")
destination_filename = f"{notebook_name}.pt"
destination_path = os.path.join(destination_dir, destination_filename)

# Copy the model file to Google Drive
if os.path.exists(source_path):
    shutil.copy(source_path, destination_path)
    print(f"✅ Trained model saved to: {destination_path}")
else:
    print(f"❌ Source model not found at {source_path}. Make sure training has finished successfully.")

# Evaluate fine-tuned YOLOv12 model

### Inspect Training Output Directory

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!ls {HOME}/runs/detect/train/

### Display Confusion Matrix

In [ ]:
from IPython.display import Image

Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=1000)

### Display Training Results Plot

In [ ]:
from IPython.display import Image

Image(filename=f'{HOME}/runs/detect/train/results.png', width=1000)

### Load Test Dataset and Display Classes

In [ ]:
import supervision as sv

ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{DATASET_PATH}/test/images",
    annotations_directory_path=f"{DATASET_PATH}/test/labels",
    data_yaml_path=DATA_YAML_PATH
)

ds.classes

### Calculate Mean Average Precision (mAP)

In [ ]:
from supervision.metrics import MeanAveragePrecision

model = YOLO(f'{HOME}/runs/detect/train/weights/best.pt')

predictions = []
targets = []

for _, image, target in ds:
    results = model(image, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)

    predictions.append(detections)
    targets.append(target)

map = MeanAveragePrecision().update(predictions, targets).compute()

### Print mAP Scores

In [ ]:
print("mAP 50:95", map.map50_95)
print("mAP 50", map.map50)
print("mAP 75", map.map75)

### Plot mAP Curve

In [ ]:
map.plot()

# Run inference with fine-tuned YOLOv12 model

In [ ]:
import supervision as sv

model = YOLO(f'{HOME}/runs/detect/train/weights/best.pt')

ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{DATASET_PATH}/test/images",
    annotations_directory_path=f"{DATASET_PATH}/test/labels",
    data_yaml_path=DATA_YAML_PATH
)

In [ ]:
import random

i = random.randint(0, len(ds))

image_path, image, target = ds[i]

results = model(image, verbose=False)[0]
detections = sv.Detections.from_ultralytics(results).with_nms()

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = image.copy()
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)

# Call tắt laptop sau khi train xong

In [ ]:
import requests

WEBHOOK_URL = "https://forcepslike-lawanda-semicalcined.ngrok-free.dev/done"  # đổi URL của bạn

try:
    r = requests.post(WEBHOOK_URL)
    print("Đã gửi tín hiệu shutdown về PC!")
except Exception as e:
    print("Không gửi được webhook:", e)